In [1]:
from utils.data_preprocess import *
import pandas as pd
import numpy as np
import tensorflow as tf
import talib as ta

In [2]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')

In [4]:
def func(df):
    if (df['Trend'] == 0) or (df['Trend'] == 1 and df['Previous_Trend'] == 1) or (df['Trend'] == -1 and df['Previous_Trend'] == -1):
        return 'Hold'
    elif (df['Trend'] == 1) and (df['Previous_Trend'] == 0 or -1):
        return 'Buy'
    elif (df['Trend'] == -1) and (df['Previous_Trend'] == 0 or 1):
        return 'Sell'

In [5]:
# DataLabeling
data = df.set_index('Timestamp').iloc[:, :5]
data['STD'] = data.Close.rolling(24).std()
data['Next_Close'] = data.Close.shift(-24)
data = data.dropna()
data['Trend'] = np.where(data.Next_Close >= data.Close*(1+0.55*data.STD), 1,
                         np.where(data.Next_Close <= data.Close*(1-0.55*data.STD), -1, 0))
data['Previous_Trend'] = data.Trend.shift(fill_value=0)
data = data.drop(['STD'], axis=1)
# data['Next_Trend'] = data.Next_Trend.astype(int)
data = data.assign(Label=data.apply(func, axis=1))
data.head()

,Open,High,Low,Close,Volume,Next_Close,Trend,Previous_Trend,Label
Timestamp,,,,,,,,,
2017-07-14 05:55:00,0.085506,0.085782,0.085341,0.085399,13.840,0.088591,1,0,Buy
2017-07-14 06:00:00,0.085399,0.085460,0.085398,0.085460,0.606,0.088591,1,1,Hold
2017-07-14 06:05:00,0.085470,0.086204,0.085402,0.086186,101.722,0.088591,1,1,Hold
2017-07-14 06:10:00,0.086274,0.087001,0.086274,0.086973,59.807,0.088591,1,1,Hold
2017-07-14 06:15:00,0.087097,0.087097,0.086898,0.086899,28.800,0.086252,-1,1,Sell


In [3]:
raw_data = DataLabeling(df, 24)

In [4]:
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data)

c:\Users\dddru\Documents\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\data_preprocess.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  np.where(data.Next_Close <= data.Close*(1-self.__alpha*data.STD), 2, 0))
c:\Users\dddru\Documents\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\data_preprocess.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Previous_Trend'] = data.Trend.shift(fill_value=0)


In [5]:
Data = DataPreprocess(train_df, val_df, test_df, window_size=24, label_size=1, label_columns=['Label'], shift=0)

In [6]:
Data.train

<MapDataset shapes: ((None, 24, 5), (None, 1, 1)), types: (tf.float32, tf.float32)>

In [7]:
from tensorflow.keras import layers, Sequential, Model, Input
import tensorflow as tf

In [8]:
class StrongModel:
    def __init__(self, window_size, features):
        self.window_size = window_size
        self.features = features
        
    def __CNN_blocks(self, id, filters, kernel_size, pool_size, pool_strides):
        block = tf.keras.Sequential([
            tf.keras.layers.Convolution2D(filters=filters, kernel_size=(1, kernel_size), strides=1, padding='same'),
            tf.keras.layers.MaxPool2D(pool_size=(1, pool_size), strides=(1, pool_strides), padding='valid')
        ], name=f'feature_extractor_{id}')
    
        return block
    
    def train(self, train_data, epochs, val_data):
        self.model.fit(train_data,
                       epochs=epochs,
                       steps_per_epoch=len(train_data),
                       validation_data=val_data,
                       validation_steps=int(0.15*len(val_data)),
                       use_multiprocessing=True,
                       callbacks=[tf.keras.callbacks.EarlyStopping(
                       monitor='val_loss', verbose=1, patience=15, restore_best_weights=True),
                                  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5)])
    
    @property
    def model(self):
        input = tf.keras.Input(shape=(self.window_size, self.features), name='input_layer')
        x = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=3), name='add_dim')(input)
        x = tf.keras.layers.Permute((2, 1, 3), name='feature_time_transpose')(x)
        x = self.__CNN_blocks(id=1, filters=32, kernel_size=4, pool_size=4, pool_strides=4)(x)
        x = self.__CNN_blocks(id=2, filters=64, kernel_size=3, pool_size=3, pool_strides=3)(x)
        x = self.__CNN_blocks(id=3, filters=128, kernel_size=2, pool_size=2, pool_strides=2)(x)
        x = tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=2), name='reduce_dim')(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(1000, activation='relu')(x)
        x = tf.keras.layers.Dense(500, activation='relu')(x)
        output = tf.keras.layers.Dense(3, activation='softmax')(x)
        model = tf.keras.Model(input, output, name='strong_baseline')

        # Compile the model
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        return model

In [10]:
test = StrongModel(24, 5)

In [11]:
test.train(Data.train, 500, Data.val)

Epoch 1/500
2481/2481 [==============================] - 28s 9ms/step - loss: 0.4437 - accuracy: 0.8876 - val_loss: 0.4111 - val_accuracy: 0.8973 - lr: 0.0010
Epoch 2/500
2481/2481 [==============================] - 24s 10ms/step - loss: 0.4324 - accuracy: 0.8880 - val_loss: 0.4042 - val_accuracy: 0.8974 - lr: 0.0010
Epoch 3/500
2481/2481 [==============================] - 22s 9ms/step - loss: 0.4337 - accuracy: 0.8880 - val_loss: 0.4047 - val_accuracy: 0.8974 - lr: 0.0010
Epoch 4/500
2481/2481 [==============================] - 22s 9ms/step - loss: 0.4302 - accuracy: 0.8881 - val_loss: 0.4007 - val_accuracy: 0.8975 - lr: 0.0010
Epoch 5/500
2481/2481 [==============================] - 23s 9ms/step - loss: 0.4295 - accuracy: 0.8881 - val_loss: 0.4020 - val_accuracy: 0.8975 - lr: 0.0010
Epoch 6/500
2481/2481 [==============================] - 22s 9ms/step - loss: 0.4279 - accuracy: 0.8881 - val_loss: 0.4035 - val_accuracy: 0.8975 - lr: 0.0010
Epoch 7/500
2481/2481 [======================

In [7]:
from tensorflow.keras import Input, layers, Model, Sequential

# Create CNN blocks for feature extraction

def CNN_blocks(id, filters, kernel_size, pool_size, pool_strides):
    block = Sequential([
        layers.Convolution2D(filters=filters, kernel_size=(1, kernel_size), strides=1, padding='same'),
        layers.MaxPool2D(pool_size=(1, pool_size), strides=(1, pool_strides), padding='valid')
    ], name=f'feature_extractor_{id}')
    
    return block

In [12]:
# Create model using Model API
input = Input(shape=(24, 6), name='input_layer')
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=3), name='add_dim')(input)
x = layers.Permute((2, 1, 3), name='feature_time_transpose')(x)
x = CNN_blocks(id=1, filters=32, kernel_size=4, pool_size=4, pool_strides=4)(x)
x = CNN_blocks(id=2, filters=64, kernel_size=3, pool_size=3, pool_strides=3)(x)
x = CNN_blocks(id=3, filters=128, kernel_size=2, pool_size=2, pool_strides=2)(x)
x = layers.Lambda(lambda x: tf.squeeze(x, axis=2), name='reduce_dim')(x)
print(x.shape)
x = layers.Flatten()(x)
print(x.shape)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dense(500, activation='relu')(x)
output = layers.Dense(3, activation='softmax')(x)
model = Model(input, output, name='strong_baseline')

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

(None, 6, 128)
(None, 768)
